In [ ]:
from Bio import SeqIO

Count Reads

In [ ]:
count = 0
for rec in SeqIO.parse("R1.fastqsanger", "fastq"):
    count += 1
print("%i reads" % count)

Filter Reads for a minimum PHRED quality of 20

In [ ]:
good_reads = (
    rec
    for rec in SeqIO.parse("R1.fastqsanger", "fastq")
    if min(rec.letter_annotations["phred_quality"]) >= 20
)
count = SeqIO.write(good_reads, "good_quality.fastq", "fastq")
print("Saved %i reads" % count)

Use Pylab or Matplotlib module to plot the Phred quality scores of the Reads

In [ ]:
import pylab

for subfigure in [1, 2]:
    filename = "R%i.fastqsanger" % subfigure
    pylab.subplot(1, 2, subfigure)
    for i, record in enumerate(SeqIO.parse(filename, "fastq")):
        if i >= 90:
            break  
        pylab.plot(record.letter_annotations["phred_quality"])
    pylab.ylim(0, 45)
    pylab.ylabel("PHRED quality score")
    pylab.xlabel("Position")
pylab.savefig("R.png")
print("Done")


Find Reads with Primer Sequences

In [ ]:
primer_reads = (
    rec
    for rec in SeqIO.parse("R1.fastqsanger", "fastq")
    if rec.seq.startswith("GATGACGGTGT")
)
count = SeqIO.write(primer_reads, "with_primer.fastq", "fastq")
print("Saved %i reads" % count)


That should find reads from R1.fastq and save them to a new FASTQ file, with_primer.fastq.

Now How to Trim these Primers..???

In [ ]:
#We are creating a funtion to trim the primers
def trim_primers(records, primer):
    """Removes perfect primer sequences at start of reads.

    This is a generator function, the records argument should
    be a list or iterator returning SeqRecord objects.
    """
    len_primer = len(primer)  # cache this for later
    for record in records:
        if record.seq.startswith(primer):
            yield record[len_primer:]
        else:
            yield record

In [ ]:
#Usage of the above Function
original_reads = SeqIO.parse("R1.fastqsanger", "fastq")
trimmed_reads = trim_primers(original_reads, "GATGACGGTGT")
count = SeqIO.write(trimmed_reads, "trimmed.fastq", "fastq")
print("Saved %i reads" % count)

Trimming of Adapter Sequences ..!!

In [ ]:
def trim_adaptors(records, adaptor):
    """Trims perfect adaptor sequences.

    This is a generator function, the records argument should
    be a list or iterator returning SeqRecord objects.
    """
    len_adaptor = len(adaptor)  # cache this for later
    for record in records:
        index = record.seq.find(adaptor)
        if index == -1:
            # adaptor not found, so won't trim
            yield record
        else:
            # trim off the adaptor
            yield record[index + len_adaptor :]

In [ ]:
original_reads = SeqIO.parse("R1.fastqsanger", "fastq")
trimmed_reads = trim_adaptors(original_reads, "GATGACGGTGT")
count = SeqIO.write(trimmed_reads, "trimmed_adapters.fastq", "fastq")
print("Saved %i reads" % count)